<a href="https://colab.research.google.com/github/rajaranjith/HCL-GenAI-Training/blob/main/Assignment_1_Langgraph_based_Automotive_Telemetry_System_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langgraph langchain_community faiss-cpu langchain-google-genai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
from google.colab import userdata
from dotenv import load_dotenv
load_dotenv()

# Get the API key from environment variables
google_api_key = userdata.get("GOOGLE_API_KEY")
import os
os.environ["GOOGLE_API_KEY"] = google_api_key

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

# Chat model (Gemini)
# Model IDs are listed in Google's Gemini API docs; gemini-2.5-pro is a stable model code. :contentReference[oaicite:2]{index=2}
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0
)

# Embeddings model
# LangChain docs show using "models/gemini-embedding-001" for embeddings. :contentReference[oaicite:3]{index=3}
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001"
)

In [4]:
#from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

#from langchain.schema import Document

docs = [
    Document(page_content="High engine temperature may indicate coolant leak or radiator failure."),
    Document(page_content="Low battery voltage often suggests battery degradation."),
    Document(page_content="Brake pressure loss is usually caused by hydraulic fluid leakage."),
    Document(page_content="Brake system faults are safety critical and require immobilization.")
]

vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [5]:
from typing import TypedDict, List, Dict

class VehicleState(TypedDict):
    vehicle_id: str
    telemetry: Dict[str, float]
    anomaly: str
    retrieved_docs: List[str]
    diagnosis: str
    decision: str


In [6]:
# Helper to robustly extract text from LangChain message objects
# (Gemini responses can be structured; .text is the safest when available). :contentReference[oaicite:4]{index=4}
def msg_to_text(msg) -> str:
    if hasattr(msg, "text") and msg.text is not None:
        return msg.text
    content = getattr(msg, "content", msg)
    if isinstance(content, str):
        return content
    if isinstance(content, list):
        parts = []
        for block in content:
            if isinstance(block, dict) and "text" in block:
                parts.append(block["text"])
            elif isinstance(block, str):
                parts.append(block)
        return "".join(parts).strip()
    return str(content)

In [7]:
def telemetry_node(state: VehicleState):
    return state

In [8]:
def anomaly_node(state: VehicleState):
    t = state["telemetry"]

    if t["engine_temp"] > 110:
        state["anomaly"] = "High engine temperature"
    elif t["battery_voltage"] < 11.5:
        state["anomaly"] = "Low battery voltage"
    elif t["brake_pressure"] < 20:
        state["anomaly"] = "Brake pressure loss"
    else:
        state["anomaly"] = "Normal"

    return state

In [9]:
def retrieval_node(state: VehicleState):
    docs = retriever.invoke(state["anomaly"])
    state["retrieved_docs"] = [d.page_content for d in docs]
    return state

In [10]:
def diagnosis_node(state: VehicleState):
    context = "\n".join(state["retrieved_docs"])

    prompt = f"""
    You are an automotive diagnostics expert.

    Telemetry:
    {state["telemetry"]}

    Detected anomaly:
    {state["anomaly"]}

    Relevant knowledge:
    {context}

    Identify the most likely root cause.
    """

    response = llm.invoke(prompt)
    state["diagnosis"] = msg_to_text(response)
    return state

In [11]:
def decision_node(state: VehicleState):
    prompt = f"""
    You are a fleet safety AI.

    Diagnosis:
    {state["diagnosis"]}

    Decide the safest action.
    Constraints:
    - Human safety first
    - Vehicle protection second
    - Cost last

    Respond with ONE action.
    """

    response = llm.invoke(prompt)
    state["decision"] = msg_to_text(response)
    return state

In [12]:
def report_node(state: VehicleState):
    print("\n===== VEHICLE INCIDENT REPORT =====")
    print("Vehicle ID:", state["vehicle_id"])
    print("Telemetry:", state["telemetry"])
    print("Anomaly:", state["anomaly"])
    print("Diagnosis:", state["diagnosis"])
    print("Decision:", state["decision"])
    print("=================================\n")

    return state


In [13]:
from langgraph.graph import StateGraph, END

graph = StateGraph(VehicleState)

graph.add_node("telemetry", telemetry_node)
graph.add_node("anomaly", anomaly_node)
graph.add_node("retrieve", retrieval_node)
graph.add_node("diagnose", diagnosis_node)
graph.add_node("decide", decision_node)
graph.add_node("report", report_node)

graph.set_entry_point("telemetry")

graph.add_edge("telemetry", "anomaly")
graph.add_edge("anomaly", "retrieve")
graph.add_edge("retrieve", "diagnose")
graph.add_edge("diagnose", "decide")
graph.add_edge("decide", "report")
graph.add_edge("report", END)

app = graph.compile()

In [14]:
initial_state = {
    "vehicle_id": "CAR-9001",
    "telemetry": {
        "engine_temp": 128,
        "battery_voltage": 12.1,
        "brake_pressure": 30
    },
    "anomaly": "",
    "retrieved_docs": [],
    "diagnosis": "",
    "decision": ""
}

app.invoke(initial_state)


ChatGoogleGenerativeAIError: Error calling model 'gemini-2.5-pro' (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.5-pro\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.5-pro\nPlease retry in 17.531638634s.', 'status': 'RESOURCE_EXHAUST